In [0]:
dbutils.fs.cp("file:/data/input/bad_records.csv", "dbfs:/data/input/bad_records.csv")

Out[37]: True

In [0]:
import os

# Create the directory if it doesn't exist
os.makedirs("/data/input/", exist_ok=True)

# Sample data with a malformed row (bad_row has only one column)
data = """id,name,age
1,Alice,30
2,Bob,not_a_number
3,,25
4,Charlie,40
bad_row,extra,columns,here
5,David,
"""

# Save sample data to a file (this file will contain a malformed row)
with open("/data/input/bad_records.csv", "w") as f:
    f.write(data)

In [0]:
df = spark.read.option("header", True).option("inferSchema", True).csv("/data/input/bad_records.csv")
df.show()

+-------+-------+------------+
|     id|   name|         age|
+-------+-------+------------+
|      1|  Alice|          30|
|      2|    Bob|not_a_number|
|      3|   null|          25|
|      4|Charlie|          40|
|bad_row|   null|        null|
|      5|  David|        null|
+-------+-------+------------+



### Permissive Mode

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Define schema for reading (strict: expects 3 columns)
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True)
])


In [0]:

df1 = spark.read.option("header", True) \
    .option("mode", "PERMISSIVE") \
    .option("columnNameOfCorruptRecord", "_corrupt_record") \
    .schema(schema) \
    .csv("/data/input/bad_records.csv")

df1.show(truncate=False)



+----+-------+----+
|id  |name   |age |
+----+-------+----+
|1   |Alice  |30  |
|2   |Bob    |null|
|3   |null   |25  |
|4   |Charlie|40  |
|null|null   |null|
|5   |David  |null|
+----+-------+----+



### Dropmalformed Mode

In [0]:

df2 = spark.read.option("header", True) \
    .option("mode", "DROPMALFORMED") \
    .schema(schema) \
    .csv("/data/input/bad_records.csv")

df2.show(truncate=False)



+---+-------+----+
|id |name   |age |
+---+-------+----+
|1  |Alice  |30  |
|3  |null   |25  |
|4  |Charlie|40  |
|5  |David  |null|
+---+-------+----+



### Failfast Mode

In [0]:

df3 = spark.read.option("header", True) \
    .option("mode", "FAILFAST") \
    .schema(schema) \
    .csv("/data/input/bad_records.csv")

df3.show(truncate=False)



---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-6336637188382304>:6
      1 df3 = spark.read.option("header", True) \
      2     .option("mode", "FAILFAST") \
      3     .schema(schema) \
      4     .csv("/data/input/bad_records.csv")
----> 6 df3.show(truncate=False)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:933, in DataFrame.show(self, n, truncate, vertical)
    924 except ValueError:
    925     raise PySparkTypeError(
    926         error_class="NOT_A_BOOLEAN",
    927         messag

> SUMMARY


**What to do:**
- If you want to see which row is bad, use `PERMISSIVE` mode with columnNameOfCorruptRecord.
- If you want Spark to skip bad rows, use `DROPMALFORMED` mode.
- `FAILFAST` is useful for strict validation, but it will always abort on the first malformed row. In `FAILFAST` mode, Spark aborts the job as soon as it finds such a row.